In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vqa-by-me/__results__.html
/kaggle/input/vqa-by-me/__resultx__.html
/kaggle/input/vqa-by-me/__notebook__.ipynb
/kaggle/input/vqa-by-me/__output__.json
/kaggle/input/vqa-by-me/custom.css
/kaggle/input/mini-coco2014-dataset-for-image-captioning/captions.json
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000487632.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000203982.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000447520.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000316443.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000464415.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000368731.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images/COCO_train2014_000000462131.jpg
/kaggle/input/mini-coco2014-dataset-for-image-captioning/

In [2]:
import os
import json
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, CLIPProcessor, CLIPVisionModel

# Paths of uploaded dataset files
IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"

# using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Verified Device: {device}")

# # Paths based on your uploaded Kaggle screenshot
# IMG_DIR = "/kaggle/input/mini-coco2014-dataset-for-image-captioning/Images"
# QUEST_PATH = "/kaggle/input/vqatext/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
# ANNOT_PATH = "/kaggle/input/vqatext/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"

2025-12-29 15:37:56.452701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767022676.676051      23 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767022676.745222      23 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767022677.319450      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767022677.319483      23 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767022677.319486      23 computation_placer.cc:177] computation placer alr

Verified Device: cuda


In [3]:

class MiniVQADataset(Dataset):
    def __init__(self, questions_path, annotations_path, img_dir, tokenizer, processor):
        # 1. Load all data
        with open(questions_path, 'r') as f:
            all_questions = json.load(f)['questions']
        with open(annotations_path, 'r') as f:
            self.annotations = {a['question_id']: a for a in json.load(f)['annotations']}
        
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.processor = processor
        # self.ans_to_id = {"yes": 0, "no": 1, "2": 2, "1": 3, "white": 4}

        # --- NEW CODE STARTS HERE: Create a dynamic Answer Vocabulary ---
        # Get all answers from the annotations you actually have
        all_raw_answers = [a['multiple_choice_answer'] for a in self.annotations.values()]
        
        # Get the 1000 most frequent answers (this is standard VQA research practice)
        most_common = Counter(all_raw_answers).most_common(1000)
        self.ans_to_id = {ans[0]: i for i, ans in enumerate(most_common)}
        

        # 2. FILTERING LOGIC: Only keep questions if the image exists on disk
        self.valid_questions = []
        print("Filtering dataset for existing images... please wait.")
        
        for q in all_questions:
            img_id = str(q['image_id']).zfill(12)
            img_path = os.path.join(self.img_dir, f"COCO_train2014_{img_id}.jpg")
            
            if os.path.exists(img_path):
                self.valid_questions.append(q)
            
            # Stop once we have 2000 valid samples to save time
            if len(self.valid_questions) >= 10000:
                break
                
        print(f"Done! Found {len(self.valid_questions)} matching images in your mini dataset.")

    def __len__(self):
        return len(self.valid_questions)

    def __getitem__(self, idx):
        q_data = self.valid_questions[idx]
        q_id = q_data['question_id']
        img_id = str(q_data['image_id']).zfill(12)
        
        img_path = os.path.join(self.img_dir, f"COCO_train2014_{img_id}.jpg")
        image = Image.open(img_path).convert("RGB")
        
        text_inputs = self.tokenizer(q_data['question'], padding='max_length', max_length=32, truncation=True, return_tensors="pt")
        image_inputs = self.processor(images=image, return_tensors="pt")
        
        ans_str = self.annotations[q_id]['multiple_choice_answer']
        label = self.ans_to_id.get(ans_str, 0) 

        return {
            'input_ids': text_inputs['input_ids'].squeeze(),
            'attention_mask': text_inputs['attention_mask'].squeeze(),
            'pixel_values': image_inputs['pixel_values'].squeeze(),
            'label': torch.tensor(label)
        }

In [4]:
class VQAModel(torch.nn.Module):
    def __init__(self, num_answers=1000):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.clip = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
        
        # BERT (768) + CLIP (768) = 1536 features
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(1536, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, num_answers)
        )

    def forward(self, input_ids, attention_mask, pixel_values):
        # Extract Text Features
        text_features = self.bert(input_ids, attention_mask=attention_mask).pooler_output
        # Extract Image Features
        image_features = self.clip(pixel_values).pooler_output
        # Fuse and Classify
        fused = torch.cat((text_features, image_features), dim=1)
        return self.classifier(fused)

In [5]:
import torch
from torch.utils.data import DataLoader, random_split
from transformers import BertTokenizer, CLIPProcessor
from tqdm import tqdm  # This adds a visual progress bar
from collections import Counter


# 1. Initialize Components
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


# 1. Initialize with 3000 samples
# This will look through your JSON and stop once it finds 3000 images on your disk
dataset_full = MiniVQADataset(QUEST_PATH, ANNOT_PATH, IMG_DIR, tokenizer, processor)
# Inside your class, change 'if len(self.valid_questions) >= 2000' to 3000

# 2. Define the split (80% for training, 20% for testing)
train_count = int(0.8 * len(dataset_full)) # 2400 samples
test_count = len(dataset_full) - train_count # 600 samples

# 3. Perform the split
# random_split gives you two subsets of your data
train_ds, test_ds = random_split(dataset_full, [train_count, test_count])



# Create Dataset and Loader
# train_ds = MiniVQADataset(QUEST_PATH, ANNOT_PATH, IMG_DIR, tokenizer, processor)
# loader = DataLoader(train_ds, batch_size=8, shuffle=True)
# 4. Create separate DataLoaders
# The model only "sees" the train_loader during training
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

# 2. Setup Model & Optimizer
# model = VQAModel(num_answers=1000).to(device) # num_answers should match your class output

# 2. Initialize Model with your new 1000-answer limit
model = VQAModel(num_answers=1000)

# 3. ACTIVATE BOTH GPUs
if torch.cuda.device_count() > 1:
    print(f"🔥 Using {torch.cuda.device_count()} GPUs for training!")
    model = torch.nn.DataParallel(model)

# 4. Move the model to the device
model.to(device)

# opt = torch.optim.AdamW(model.parameters(), lr=5e-5)
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = torch.nn.CrossEntropyLoss()

# 3. Merged Training Loop with Accuracy
# NUM_EPOCHS = 1 # Start with 1 to test
NUM_EPOCHS = 40 # Start with 1 to test
model.train()

for epoch in range(NUM_EPOCHS):
    running_corrects = 0
    total_samples = 0
    
    # tqdm creates a progress bar so you don't just see a static number
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    
    for batch in progress_bar:
        # Move batch to GPU
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        pixels = batch['pixel_values'].to(device)
        labels = batch['label'].to(device)
        
        # Forward Pass
        # opt.zero_grad()
        opt.zero_grad(set_to_none=True)
        outputs = model(ids, mask, pixels)
        loss = loss_fn(outputs, labels)
        
        # --- Accuracy Calculation ---
        _, preds = torch.max(outputs, 1)            # Get the predicted answer ID
        running_corrects += (preds == labels).sum().item() # Count matches
        total_samples += labels.size(0)             # Count total items
        current_acc = 100 * running_corrects / total_samples
        # ----------------------------

        # Backward Pass
        loss.backward()
        opt.step()
        
        # Update progress bar with Loss and Accuracy
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}', 
            'acc': f'{current_acc:.2f}%'
        })

# 4. Save Final Weights
# torch.save(model.state_dict(), "/kaggle/working/vqa_mini.pth")
# after using 2 GPUs
# Use .module to get the original model inside the wrapper
torch.save(model.module.state_dict(), "/kaggle/working/vqa_mini.pth")

print(f"\nTraining Finished! Final Accuracy: {current_acc:.2f}%")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

🔥 Using 2 GPUs for training!


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]


Epoch 40: 100%|██████████| 125/125 [02:48<00:00,  1.35s/it, loss=0.1666, acc=94.34%]



Training Finished! Final Accuracy: 94.34%


In [6]:
# Save the weights safely
if isinstance(model, torch.nn.DataParallel):
    # This reaches inside the multi-GPU wrapper to get the actual weights
    torch.save(model.module.state_dict(), "/kaggle/working/vqa_best_97.pth")
    print("Multi-GPU weights saved successfully!")
else:
    torch.save(model.state_dict(), "/kaggle/working/vqa_best_97.pth")
    print("Single-GPU weights saved successfully!")

Multi-GPU weights saved successfully!


# Testing 

In [7]:
model.eval()  # Set model to evaluation mode
test_running_corrects = 0
test_total_samples = 0

print("Starting Evaluation on Test Set...")

# Disable gradient calculations for speed and memory efficiency
with torch.no_grad():
    # Use tqdm to monitor progress on the 600 test samples
    test_progress_bar = tqdm(test_loader, desc="Testing")
    
    for batch in test_progress_bar:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        pixels = batch['pixel_values'].to(device)
        labels = batch['label'].to(device)

        # Forward Pass
        outputs = model(ids, mask, pixels)
        
        # Calculate Accuracy
        _, preds = torch.max(outputs, 1)
        test_running_corrects += (preds == labels).sum().item()
        test_total_samples += labels.size(0)
        
        # Update progress bar
        current_test_acc = 100 * test_running_corrects / test_total_samples
        test_progress_bar.set_postfix({'test_acc': f'{current_test_acc:.2f}%'})

final_test_accuracy = 100 * test_running_corrects / test_total_samples
print(f"\n--- Evaluation Results ---")
print(f"Final Test Accuracy: {final_test_accuracy:.2f}%")
print(f"Training Accuracy was: {current_acc:.2f}%") # This is from your training loop
print(f"The 'Generalization Gap' is: {current_acc - final_test_accuracy:.2f}%")

Starting Evaluation on Test Set...


Testing: 100%|██████████| 63/63 [00:29<00:00,  2.14it/s, test_acc=36.75%]


--- Evaluation Results ---
Final Test Accuracy: 36.75%
Training Accuracy was: 94.34%
The 'Generalization Gap' is: 57.59%


In [8]:
model.eval()
test_running_corrects = 0
test_total_samples = 0

print(f"Starting Evaluation on {len(test_ds)} images...")

with torch.no_grad():
    # We set total=len(test_ds) and update by the batch size manually
    test_progress_bar = tqdm(total=len(test_ds), desc="Testing Images")
    
    for batch in test_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        pixels = batch['pixel_values'].to(device)
        labels = batch['label'].to(device)

        outputs = model(ids, mask, pixels)
        
        _, preds = torch.max(outputs, 1)
        batch_corrects = (preds == labels).sum().item()
        test_running_corrects += batch_corrects
        test_total_samples += labels.size(0)
        
        # Update bar by number of images in this batch (usually 8)
        test_progress_bar.update(labels.size(0))
        
        current_test_acc = 100 * test_running_corrects / test_total_samples
        test_progress_bar.set_postfix({'test_acc': f'{current_test_acc:.2f}%'})

    test_progress_bar.close()

print(f"\nFinal Test Accuracy: {current_test_acc:.2f}%")

Starting Evaluation on 2000 images...


Testing Images: 100%|██████████| 2000/2000 [00:29<00:00, 68.61it/s, test_acc=36.75%]


Final Test Accuracy: 36.75%
